# Paper replication

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
from utils import all_combinations

M = 2
D = 3 * 3
ALL_INPUTS = all_combinations(M, D)

## Define CAs and training data

In [ ]:
from ca_funcs import make_table_walk, make_ca

def sample_CAs(seed=None):
    if seed:
        np.random.seed(seed)
        
    inputs = ALL_INPUTS
    outputs = make_table_walk(len(ALL_INPUTS))
    for o in outputs:
        yield make_ca(inputs, o)
        
def generate_CA_train_data(ca, height=10, width=10, n_samples=500):
    X_train = np.random.choice([0,1], (n_samples, height, width), p=[.5,.5])
    Y_train = ca(tf.convert_to_tensor(X_train, tf.float32))
    return (X_train, Y_train)

In [ ]:
from ca_funcs import make_glider
from IPython.display import clear_output

np.random.seed(0)

for i, ca in enumerate(sample_CAs()):
    X_test = tf.convert_to_tensor(make_glider(10).reshape(1, 10, 10), tf.float32)
    Y_test = ca(X_test)
    
    plt.figure(figsize=(12,4))
    plt.suptitle(i)

    plt.subplot(1,2,1)
    plt.imshow(X_test[0])
    plt.axis('off')
    plt.title("Input")

    plt.subplot(1,2,2)
    plt.imshow(Y_test[0])
    plt.axis('off')
    plt.title("Output")
    
    plt.show()
    plt.close()
    clear_output(wait=True)

## Find entropy of the training CA

In [ ]:
from collections import Counter
from utils import shannon_entropy

def ca_entropy(ca):
    inputs = tf.convert_to_tensor(ALL_INPUTS, tf.float32)
    outputs = ca(inputs)
    output_counts = np.array(list(Counter(tuple(tf.reshape(o, [-1]).numpy()) for o in outputs).values()))
    output_ps = output_counts / len(inputs)
    return shannon_entropy(output_ps)

In [ ]:
entropies = [ca_entropy(a) for a in tqdm(sample_CAs(seed=0))]
plt.plot(entropies)

## Define the model

In [ ]:
from train_ca import initialize_model

seed = 0
print('seed =', seed)

np.random.seed(seed)
tf.random.set_seed(seed)

num_classes = 2
samples = 500
input_dims = [10, 10]
layer_dims = [100] + [100] * 11 # neighborhood conv + mlpconv layers
batch_size = 10
num_batches = samples / batch_size
learning_rate = 1e-4
training_epochs = 100 # 1500
display_step = int(training_epochs/10)
loss = tf.keras.losses.MeanSquaredError()
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)

model = initialize_model(input_dims, layer_dims)
model.compile(optimizer=optimizer, loss=loss)
model.summary()

## Define the learning loop

In [ ]:
def learn_CA(ca, model):
    X_train, Y_train = generate_CA_train_data(ca, *input_dims, n_samples=samples)
    model.fit(x=X_train, y=Y_train, epochs=training_epochs, batch_size=batch_size)

## Train the model

In [ ]:
ca = list(sample_CAs(seed=0))[250]
learn_CA(ca, model)

In [ ]:
plt.figure(figsize=(10, 5))
plt.subplot(121)
plt.plot(model.history.history['loss'])
plt.subplot(122)
plt.plot(model.history.history['loss'])
plt.loglog()

In [ ]:
from ca_funcs import make_glider

# x = np.random.choice([0, 1], size=100)
x = make_glider(10)
X_test = tf.convert_to_tensor(x.reshape(1, 10, 10), tf.float32)
Y_test = ca(X_test)
Y_pred = model(X_test)

plt.figure(figsize=(12,4))

plt.subplot(141)
plt.imshow(X_test[0])
plt.axis('off')
plt.title("Input")

plt.subplot(142)
plt.imshow(Y_test[0])
plt.axis('off')
plt.title("Expected Output")

plt.subplot(143)
plt.imshow(Y_pred[0])
plt.axis('off')
plt.title("Observed Output")

plt.subplot(144)
plt.imshow((Y_pred[0] - Y_test[0])**2)
plt.axis('off')
plt.title("Diff")

In [ ]:
tf.reduce_max(tf.abs(Y_pred[0] - Y_test[0]))

## Find model entropies

In [ ]:
from ca_funcs import get_network_entropies

get_activations = tf.keras.backend.function(model.input, [model.layers[i].output for i in np.r_[1, 3:len(model.layers)-2]])

X_test = tf.convert_to_tensor(np.random.choice([0, 1], (500, 10, 10)), tf.float32)

layer_activations = np.array([layer.reshape(500, 100, 100) for layer in get_activations(X_test)])
binary_activations = np.digitize(layer_activations, [0], right=True)
entropies = get_network_entropies(binary_activations)

In [ ]:
plt.plot(np.array(entropies[2]).mean(axis=1), 'o')

In [ ]:
ca_entropy(ca)

In [ ]:
entropies